In [1]:
import os
import pandas as pd

# Function to read CSV files from each folder
def read_csv_from_folders(base_path):
    data_frames = []
    for folder_name in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder_name)
        if os.path.isdir(folder_path):
            csv_path = os.path.join(folder_path, 'final_design_stats.csv')
            if os.path.exists(csv_path):
                df = pd.read_csv(csv_path)
                data_frames.append(df)
    return data_frames

# Example usage
base_path = './out/bindcraft/'
data_frames = read_csv_from_folders(base_path)

# Display the first dataframe as an example
if data_frames:
    data_frames[0].head()
else:
    print("No CSV files found.")

In [2]:
combined_df = pd.concat(data_frames, ignore_index=True)
combined_df.head()

,Rank,Design,Protocol,Length,Seed,Helicity,Target_Hotspot,Sequence,InterfaceResidues,MPNN_score,...,1_Binder_RMSD,2_Binder_RMSD,3_Binder_RMSD,4_Binder_RMSD,5_Binder_RMSD,DesignTime,Notes,TargetSettings,Filters,AdvancedSettings
0,1,5nq4_l120_s808237_mpnn5,4stage,120,808237,-0.3,NaN,MNVDLTEEKIREALEKDPRDAMILFMKAFHALRGDDEKIVEVFEKF...,"B22,B23,B26,B27,B29,B30,B33,B71,B74,B75,B77,B7...",0.96,...,1.99,1.89,NaN,NaN,NaN,"0 hours, 1 minutes, 11 seconds",NaN,5nq4,default_filters,default_4stage_multimer
1,2,5nq4_l120_s808237_mpnn8,4stage,120,808237,-0.3,NaN,MDVNLTEEKIREAAEKDPREAMILFMKAFHALRGDDKGIVKVFDIA...,"B22,B23,B25,B26,B27,B29,B30,B33,B71,B74,B75,B7...",0.97,...,2.56,2.18,NaN,NaN,NaN,"0 hours, 1 minutes, 14 seconds",NaN,5nq4,default_filters,default_4stage_multimer
2,1,5nq4_l88_s194736_mpnn6,4stage,88,194736,-0.3,NaN,SMKEVEEAKRELEAVYEHVMKVLEMWRKVMPEFKEKIDELEKKIKE...,"B10,B11,B14,B15,B18,B19,B21,B22,B25,B26,B28,B2...",0.89,...,1.44,1.43,NaN,NaN,NaN,"0 hours, 1 minutes, 7 seconds",NaN,5nq4,default_filters,default_4stage_multimer
3,2,5nq4_l88_s194736_mpnn5,4stage,88,194736,-0.3,NaN,SMEEVEEAKRELEAVYEHVLKVLEMWEKVMPEKKKELEELKKEIEE...,"B7,B10,B11,B14,B15,B18,B19,B21,B22,B25,B26,B29...",0.89,...,1.08,1.03,NaN,NaN,NaN,"0 hours, 1 minutes, 7 seconds",NaN,5nq4,default_filters,default_4stage_multimer
4,3,5nq4_l86_s533102_mpnn8,4stage,86,533102,-0.3,NaN,MEFKKLSYEEVLEYLEKNKDKFPYMARQVIPVMERMKKGSEIMQKL...,"B24,B27,B28,B32,B35,B36,B43,B46,B50,B67,B68,B7...",1.00,...,1.56,1.52,NaN,NaN,NaN,"0 hours, 1 minutes, 12 seconds",NaN,5nq4,default_filters,default_4stage_multimer


In [3]:
combined_df.tail()

,Rank,Design,Protocol,Length,Seed,Helicity,Target_Hotspot,Sequence,InterfaceResidues,MPNN_score,...,1_Binder_RMSD,2_Binder_RMSD,3_Binder_RMSD,4_Binder_RMSD,5_Binder_RMSD,DesignTime,Notes,TargetSettings,Filters,AdvancedSettings
4,3,5nq4_l86_s533102_mpnn8,4stage,86,533102,-0.3,NaN,MEFKKLSYEEVLEYLEKNKDKFPYMARQVIPVMERMKKGSEIMQKL...,"B24,B27,B28,B32,B35,B36,B43,B46,B50,B67,B68,B7...",1.00,...,1.56,1.52,NaN,NaN,NaN,"0 hours, 1 minutes, 12 seconds",NaN,5nq4,default_filters,default_4stage_multimer
5,4,5nq4_l86_s533102_mpnn6,4stage,86,533102,-0.3,NaN,MPFKKLSYEEVLAYLKENKDKYPYMARQVIPVMERMKKGSEIMKKL...,"B24,B27,B28,B32,B35,B36,B40,B42,B43,B46,B50,B7...",1.00,...,1.34,1.50,NaN,NaN,NaN,"0 hours, 1 minutes, 16 seconds",NaN,5nq4,default_filters,default_4stage_multimer
6,1,5nq4_l84_s3585_mpnn4,4stage,84,3585,-0.3,NaN,TIEELEKWEPEHIVIHYEMVKETVLEEMEEEPEMADFDIEAWVEKF...,"B1,B2,B4,B5,B8,B12,B15,B16,B17,B19,B20,B23,B24...",0.88,...,1.32,1.35,NaN,NaN,NaN,"0 hours, 1 minutes, 5 seconds",NaN,5nq4,default_filters,default_4stage_multimer
7,1,5nq4_l120_s575933_mpnn1,4stage,120,575933,-0.3,NaN,MKTKEELLNEIIELSKKYYEKIKTEEVSNEEMAEMIKELMGKFIEL...,"B19,B54,B55,B57,B60,B61,B63,B64,B67,B68,B70,B7...",0.86,...,0.93,0.96,NaN,NaN,NaN,"0 hours, 2 minutes, 20 seconds",Relaxed structure contains clashes.,5nq4,default_filters,default_4stage_multimer
8,2,5nq4_l120_s575933_mpnn12,4stage,120,575933,-0.3,NaN,MKSKEELLNEIIELSKKYYEKIKTEDVSNEEMAEMIRELFGKFIEI...,"B19,B54,B55,B57,B60,B61,B63,B64,B67,B68,B71,B7...",0.91,...,0.93,0.94,NaN,NaN,NaN,"0 hours, 1 minutes, 9 seconds",NaN,5nq4,default_filters,default_4stage_multimer


In [4]:
combined_df.to_csv('combined_data.csv', index=False)